# **Regression Infrared Dataset**

## PyTorch

### Instalasi

In [ ]:
!pip install torch

### Import Library

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

### Dataset

In [ ]:
!wget -O Infrared.csv "https://raw.githubusercontent.com/farrelrassya/teachingMLDL/main/02.%20Deep%20Learning/Dataset/Infrared.csv"
df = pd.read_csv('Infrared.csv')

# df.drop(["Gender"], axis=1, inplace=True)
# df.drop(["Age"], axis=1, inplace=True)
# df.drop(["Ethnicity"], axis=1, inplace=True)

cat_cols = ['Gender','Age','Ethnicity']
cat_cols = [col for col in cat_cols if col in df.columns]
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

df.head(5)
# df.tail(5)

--2025-03-12 20:07:21--  https://raw.githubusercontent.com/farrelrassya/teachingMLDL/main/02.%20Deep%20Learning/Dataset/Infrared.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244181 (238K) [text/plain]
Saving to: ‘Infrared.csv’

Infrared.csv        100%[===================>] 238.46K  --.-KB/s    in 0.003s  

2025-03-12 20:07:21 (72.1 MB/s) - ‘Infrared.csv’ saved [244181/244181]



,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,aveAllL13_1,T_RC1,T_RC_Dry1,...,Age_26-30,Age_31-40,Age_41-50,Age_51-60,Age_>60,Ethnicity_Asian,Ethnicity_Black or African-American,Ethnicity_Hispanic/Latino,Ethnicity_Multiracial,Ethnicity_White
0,24.0,28.0,0.8,0.7025,35.0300,35.3775,34.4000,34.9175,34.9850,34.9850,...,False,False,True,False,False,False,False,False,False,True
1,24.0,26.0,0.8,0.7800,34.5500,34.5200,33.9300,34.2250,34.7100,34.6325,...,False,True,False,False,False,False,True,False,False,False
2,24.0,26.0,0.8,0.8625,35.6525,35.5175,34.2775,34.8000,35.6850,35.6675,...,False,False,False,False,False,False,False,False,False,True
3,24.0,27.0,0.8,0.9300,35.2225,35.6125,34.3850,35.2475,35.2075,35.2000,...,False,False,False,False,False,False,True,False,False,False
4,24.0,27.0,0.8,0.8950,35.5450,35.6650,34.9100,35.3675,35.6025,35.4750,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
df.info()

In [ ]:
# print(df.isnull().sum())

imputer = IterativeImputer()
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
# print(df_imputed.isnull().sum())

### Input & Output

In [ ]:
X = df_imputed.drop(columns=['aveOralM'])
y = df_imputed['aveOralM']

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.to_numpy().reshape(-1, 1)).flatten()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)

### Model & Training

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(input_dim, 128)  # First hidden layer
        self.fc2 = nn.Linear(128, 64)         # Second hidden layer
        self.fc3 = nn.Linear(64, 32)          # Third hidden layer
        self.fc4 = nn.Linear(32, 1)           # Output layer
        self.relu = nn.ReLU()                 # ReLU activation
        self.dropout = nn.Dropout(0.2)        # Optional: add dropout for regularization

    def forward(self, x):
        # Define forward pass
        x = self.relu(self.fc1(x))            # Apply ReLU after first layer
        x = self.dropout(x)                   # Optional: apply dropout
        x = self.relu(self.fc2(x))            # Apply ReLU after second layer
        x = self.dropout(x)                   # Optional: apply dropout
        x = self.relu(self.fc3(x))            # Apply ReLU after third layer
        x = self.fc4(x)                       # No activation on output layer for regression
        return x

input_dim = X_train.shape[1]
model = NeuralNetwork(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/200], Loss: 0.7294
Epoch [20/200], Loss: 0.3673
Epoch [30/200], Loss: 0.3140
Epoch [40/200], Loss: 0.2625
Epoch [50/200], Loss: 0.2290
Epoch [60/200], Loss: 0.2141
Epoch [70/200], Loss: 0.2045
Epoch [80/200], Loss: 0.1963
Epoch [90/200], Loss: 0.1837
Epoch [100/200], Loss: 0.1778
Epoch [110/200], Loss: 0.1744
Epoch [120/200], Loss: 0.1764
Epoch [130/200], Loss: 0.1704
Epoch [140/200], Loss: 0.1655
Epoch [150/200], Loss: 0.1620
Epoch [160/200], Loss: 0.1455
Epoch [170/200], Loss: 0.1540
Epoch [180/200], Loss: 0.1466
Epoch [190/200], Loss: 0.1379
Epoch [200/200], Loss: 0.1449


### Eval

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

    # Convert back to original scale for interpretability
    y_pred_original = scaler_y.inverse_transform(y_pred.numpy())
    y_test_original = scaler_y.inverse_transform(y_test_tensor.numpy())

print()

# Calculate MSE
pytorch_mse = mean_squared_error(y_test_original, y_pred_original)

# Calculate RMSE
pytorch_rmse = np.sqrt(pytorch_mse)

# Calculate R-squared
pytorch_r2 = r2_score(y_test_original, y_pred_original)

print(f"Mean Squared Error (MSE): {pytorch_mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {pytorch_rmse:.4f}")
print(f"R-squared (R²): {pytorch_r2:.4f}")

Test Loss: 0.2728

Mean Squared Error (MSE): 0.0708
Root Mean Squared Error (RMSE): 0.2660
R-squared (R²): 0.7510


## TensorFlow

### Instalasi

In [ ]:
!pip install tensorflow

### Import Library

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### Model & Training

In [ ]:
model_tf = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

model_tf.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

es = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

mc = ModelCheckpoint(
    'best_tf_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

history = model_tf.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es,mc],
    verbose=1
)

model_tf.evaluate(X_test, y_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
17/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.6560 - mae: 0.5508
Epoch 1: val_loss improved from inf to 0.30037, saving model to best_tf_model.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - loss: 0.6527 - mae: 0.5499 - val_loss: 0.3004 - val_mae: 0.4388
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.4080 - mae: 0.4765
Epoch 2: val_loss improved from 0.30037 to 0.25304, saving model to best_tf_model.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.4067 - mae: 0.4762 - val_loss: 0.2530 - val_mae: 0.3962
Epoch 3/200
 1/18 ━━━━━━━━━━━━━━━━━━━━ 6s 391ms/step - loss: 0.2717 - mae: 0.4122
Epoch 3: val_loss improved from 0.25304 to 0.24157, saving model to best_tf_model.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.3015 - mae: 0.4159 - val_loss: 0.2416 - val_mae: 0.3927
Epoch 4/200
 1/18 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step - loss: 0.3975 - mae: 0.4685
Epoch 4: val_loss improved from 0.24157 to 0.23000, saving model to best_tf_model.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3176 - mae: 0.4126 - val_loss: 0.2300 - val_mae: 0.3731
Epoch 5/200
17/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2518 - mae: 0.3753  
Epoch 5: val_loss improved from 0.23000 to 0.22928, saving model to best_tf_model.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2520 - mae: 0.3761 - val_loss: 0.2293 - val_mae: 0.3768
Epoch 6/200
 1/18 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - loss: 0.1582 - mae: 0.3267
Epoch 6: val_loss did not improve from 0.22928
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2484 - mae: 0.3794 - val_loss: 0.2343 - val_mae: 0.3760
Epoch 7/200
 1/18 ━━━━━━━━━━━━━━━━━━━━ 3s 177ms/step - loss: 0.1481 - mae: 0.3200
Epoch 7: val_loss did not improve from 0.22928
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2300 - mae: 0.3635 - val_loss: 0.2308 - val_mae: 0.3776
Epoch 8/200
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.1770 - mae: 0.3308
Epoch 8: val_loss did not improve from 0.22928
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2198 - mae: 0.3512 - val_loss: 0.2360 - val_mae: 0.3816
Epoch 9/200
 1/18 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - loss: 0.3419 - mae: 0.3987
Epoch 9: val_loss did not improve from 0.22928
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2855 - mae: 0.3

[0.30044376850128174, 0.43049630522727966]

### Eval

In [ ]:
y_pred2 = model_tf.predict(X_test)

y_pred_original2 = scaler_y.inverse_transform(y_pred2)
y_test_original2 = scaler_y.inverse_transform(y_test.reshape(-1,1))

tf_mse = mean_squared_error(y_test_original2, y_pred_original2)
tf_rmse = np.sqrt(tf_mse)
tf_r2 = r2_score(y_test_original2, y_pred_original2)

print("\nTensorFlow Model Evaluation:")
print(f"Mean Squared Error (MSE): {tf_mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {tf_rmse:.4f}")
print(f"R-squared (R²): {tf_r2:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

TensorFlow Model Evaluation:
Mean Squared Error (MSE): 0.0779
Root Mean Squared Error (RMSE): 0.2791
R-squared (R²): 0.7258
